# Lokalna Vektorizacija
Vektorje za reprezentacijo članka (embedding) računam lokalno.
Če nimaš grafične kartice traja 100 let.

In [41]:
!pip install pandas transformers numpy torch networkx numpy scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [42]:
import os
import pandas as pd

data = pd.read_excel(os.path.join('..', 'data', 'Reworkan data 4.xlsx'))
data.head()

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Thursday,14,2023-04-06 01:31:33.498,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,bc228842-98cc-48ea-8be9-0107becb707a,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Friday,14,2023-04-07 13:47:41.072,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,"2. razred (34,5 - 59 točk)",odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Saturday,15,2023-04-15 21:44:58.961,internet,splet,bc20546f-3a11-4061-90c2-2769468cd542,Delo.si,f57a09ac-450d-4df4-bc2b-a2c7f96a4322,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Tuesday,14,2023-04-04 08:22:00.122,tisk,tisk,a67b08ee-a757-40ff-8d7e-ab8fc8246eec,Jana,4dd54f38-07e1-448f-b96d-dad85e669834,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Monday,16,2023-04-17 19:40:59.639,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."


Funkcije za vektorizacijo multilingual-e5-base nevronski model (lokalno računanje)

In [43]:
import torch
import torch.nn.functional as functional

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def e5embed(df: pd.DataFrame, col_name: str, target_col_name: str = 'Embedding'):
    def _average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    def _e5_embed(text, tokenizer, model, max_len):
        if isinstance(text, float):
            text = str(text)
        batch_dict = tokenizer(
            ['passage: ' + text], max_length=max_len,
            padding=True, truncation=True, return_tensors='pt'
        )
        batch_dict.to(device)
        outputs = model(**batch_dict)
        embeddings = _average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = functional.normalize(embeddings, p=2, dim=1)
        return embeddings.detach().cpu().numpy()[0]
    
    model_name = "intfloat/multilingual-e5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    model.to(device)
    print("Loaded model, will compute embeddings ...")
    df[target_col_name] = df[col_name].apply(_e5_embed, model=model, tokenizer=tokenizer, max_len=512)
    print("Computed embeddings")
    return df

Kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [44]:
data = e5embed(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(100)

Loaded model, will compute embeddings ...
Computed embeddings


,Article ID,Datum,Medij,Tekst,Embedding
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,"V spomin Prof. dr. Marija Pečan, dr. med. (193...","[0.022848813, 0.050237596, 0.014087035, 0.0405..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,"Člani Strateškega sveta za prehrano, ki je med...","[0.03551856, 0.073153414, -0.008020588, 0.0186..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,Člani komisije se niso opredeljevali do znakov...,"[0.026685594, 0.04836063, 0.012246748, 0.00128..."
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit...","[0.008627941, 0.038146276, -0.021161169, 0.014..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,"""Tudi vsi v zdravstvenem sistemu bodo dobili s...","[0.036248676, 0.053472724, 0.01777791, 0.01507..."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,Prodaja prehranskih dopolnil je vse bolj razši...,"[0.036037818, 0.06924387, -0.01064763, 0.00568..."
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,Raziskujemo spanje. Vsi že poznamo spalne cikl...,"[0.017657526, 0.047999874, -0.0014392067, 0.01..."
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,"Termometer prikaže, kako vroč je članek. Skupn...","[0.012317047, 0.038933747, 0.016311843, 0.0406..."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,SLOVENIJA\n'Parkinsonova bolezen v našem siste...,"[0.005352284, 0.05056906, 0.0015653971, 0.0335..."


Definiram funkcijo za clustering

In [45]:
import networkx as nx
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

Izračunam cluster

In [46]:
data = cluster_louvain(data, 'Embedding', 0.92)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data - we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)

Clustering ...
Clustered


,Article ID,Datum,Medij,Cluster,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,0,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,1,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,2,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,3,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,2,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,10,Prodaja prehranskih dopolnil je vse bolj razši...
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,96,Raziskujemo spanje. Vsi že poznamo spalne cikl...
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,15,"Termometer prikaže, kako vroč je članek. Skupn..."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,50,SLOVENIJA\n'Parkinsonova bolezen v našem siste...
